<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Практическая работа № 12. Обучение с подкреплением на основе человеческой обратной связи (RLHF): от сбора предпочтений до выравнивания языковых моделей**

## **1. Цель и задачи работы**

Цель работы — формирование у обучающегося целостного, критически рефлексивного понимания парадигмы **выравнивания (alignment)** языковых моделей с человеческими ценностями через конвейер **обучения с подкреплением на основе человеческой обратной связи (Reinforcement Learning from Human Feedback, RLHF)**. Работа направлена не только на освоение технических приёмов реализации RLHF, но и на развитие компетенций в области **этического проектирования ИИ**, **анализа смещений**, **интерпретации поведенческих сдвигов** и **обеспечения воспроизводимости** в условиях субъективной и многокритериальной оценки.

**Основные задачи работы:**  
1. На основе корпуса из Практической работы № 1 сформировать трёхкомпонентную экспериментальную базу: запросы, генерации, ранжированные предпочтения.  
2. Собрать и верифицировать **многокритериальный набор человеческих предпочтений** по трём измерениям: *полезность (helpfulness)*, *правдивость (truthfulness)*, *безопасность (harmlessness)*.  
3. Реализовать и сравнить **четыре стратегии обучения с предпочтениями**:  
 – классический RLHF с PPO,  
 – Direct Preference Optimization (DPO),  
 – Identity Preference Optimization (IPO),  
 – Cross-Entropy Method (CEM) как baseline без RL.  
4. Обучить **многоголовую модель вознаграждения (Multi-head Reward Model)**, способную раздельно оценивать каждый критерий, и исследовать возможность управления компромиссом между ними.  
5. Провести строгий мониторинг RL-процесса: KL-регуляризация, контроль дисперсии advantage, early stopping по экспертной метрике.  
6. Оценить качество моделей через **три независимых канала**:  
 – автоматические метрики (BERTScore, MAUVE, Self-BLEU),  
 – контролируемое экспертное аннотирование (Likert-шкалы, pairwise A/B),  
 – поведенческий анализ (частота льстивых фраз, лексическое разнообразие, длина, фактологическая консистентность).  
7. Проанализировать **риски переоптимизации**, **смещения аннотаторов** и **этической деформации** (например, чрезмерная политкорректность, избегание сложных тем).  
8. Разработать **интерактивную платформу для живого A/B-тестирования** с возможностью сбора онлайн-обратной связи и её интеграции в цикл дообучения (online RLHF).  
9. Обеспечить полную **воспроизводимость и открытость**: публикация набора предпочтений, RM, всех адаптированных моделей, скриптов и отчётов в соответствии с принципами FAIR и стандартами Hugging Face.  
10. Сформулировать **практические рекомендации и этические руководства** по применению RLHF в low-resource и multilingual (включая русский и татарский) контекстах.

---

## **2. Теоретические предпосылки**

RLHF возник как ответ на фундаментальное ограничение обучения с учителем (SFT): в задачах генерации не существует единственного «правильного» ответа. Вместо этого качество определяется **многомерной, субъективной и контекстуальной** оценкой. RLHF преодолевает эту неопределённость, заменяя точечную разметку **относительными предпочтениями**, которые лучше отражают сложность человеческого суждения.

### **Архитектура конвейера RLHF**
1. **Supervised Fine-Tuning (SFT)** — базовая адаптация предобученной языковой модели (например, mt5, ruGPT) к формату целевой задачи (QA, диалог, суммаризация).  
2. **Моделирование вознаграждения (Reward Modeling)** — обучение функции \( r_\theta(x, y) \), предсказывающей, насколько ответ \( y \) на запрос \( x \) соответствует человеческим предпочтениям. Обычно используется Bradley-Terry модель:  
   \[
   P(y_1 \succ y_2 | x) = \sigma(r_\theta(x, y_1) - r_\theta(x, y_2))
   \]  
3. **Обучение с подкреплением** — дообучение SFT-модели \(\pi_\phi\) с использованием алгоритма PPO, где вознаграждение заменено на \( r_\theta \), а регуляризация обеспечивается через KL-дивергенцию к reference-модели \(\pi_{\text{ref}}\):  
   \[
   \mathcal{L}_{\text{RL}} = \mathbb{E} \left[ r_\theta(x, y) - \beta \cdot \text{KL}(\pi_\phi(y|x) \| \pi_{\text{ref}}(y|x)) \right]
   \]

### **Современные альтернативы**
- **DPO** (Rafailov et al., 2023) показывает, что оптимизация предпочтений может быть сведена к задаче классификации без явного RL:  
  \[
  \mathcal{L}_{\text{DPO}} = -\log \sigma \left( \beta \log \frac{\pi_\phi(y_1|x)}{\pi_{\text{ref}}(y_1|x)} - \beta \log \frac{\pi_\phi(y_2|x)}{\pi_{\text{ref}}(y_2|x)} \right)
  \]  
- **IPO** и **SLiC** улучшают стабильность при слабых предпочтениях.

### **Критические вызовы**
- **Смещение аннотаторов**: культурные, языковые, профессиональные различия искажают предпочтения.  
- **Переоптимизация**: модель учится «обманывать» RM, генерируя формально корректные, но бессодержательные или льстивые ответы.  
- **Выравнивание vs. компетентность**: чрезмерный упор на безопасность может подавлять полезные, но спорные высказывания.  
- **Отсутствие ground truth**: качество RLHF оценивается только через прокси, что требует мульти-методного подхода.

---

## **3. Порядок выполнения работы**

### **3.1. Формирование мультизадачного набора запросов и генераций**

**Задача:** Подготовить данные для трёх типов генеративных задач на русском языке.  
**Требования:**  
– Источники: ТАСС, РИА, Meduza, belgech.ru, Татарская электронная библиотека (для татарского субкорпуса);  
– Задачи:  
 ✓ **QA**: «Ответь на вопрос по тексту» (1 000 запросов),  
 ✓ **Диалог**: «Как вежливо ответить на это сообщение?» (1 000),  
 ✓ **Суммаризация**: «Напиши кратко суть новости» (1 000);  
– Генерация: 6 вариантов на запрос с помощью SFT-модели (mt5-small, ruGPT-3.5) при температурах {0.3, 0.7, 1.0} и top-p {0.85, 0.95};  
– Формат: JSONL с `prompt`, `responses`, `task_type`;  
– Общий объём: ≥18 000 пар «запрос–ответ».

### **3.2. Сбор многокритериальных предпочтений**

**Задача:** Создать верифицированный набор ранжированных данных.  
**Требования:**  
– Критерии:  
 ✓ **Helpfulness**: решает ли задачу?  
 ✓ **Truthfulness**: соответствует ли фактам?  
 ✓ **Harmlessness**: нет ли вреда, дискриминации, дезинформации?  
– Инструкция для аннотаторов с примерами и контрольными вопросами;  
– Минимум 5 независимых оценок на пару (для оценки согласия);  
– Объём: ≥7 000 ранжированных пар (включая 1 000 на татарском);  
– Метрика согласия: Krippendorff’s α ≥ 0.65;  
– Формат: Hugging Face `PreferenceDataset` (chosen/rejected).

### **3.3. Реализация модуля RLHF с поддержкой современных методов**

**Задача:** Разработать гибкий, масштабируемый пайплайн.  
**Требования:**  
– Поддержка:  
 ✓ PPO (с библиотекой TRL или собственной реализацией),  
 ✓ DPO, IPO (через TRL или Axolotl),  
 ✓ CEM (beam search + переобучение на топ-K),  
 ✓ Multi-head RM (3 выхода: H, T, S);  
– Архитектуры:  
 ✓ Языковая модель: `cointegrated/ru-t5-base`, `sberbank-ai/ruGPT-3.5-1.7B`,  
 ✓ RM: `DeepPavlov/rubert-base-cased` с линейной головой;  
– Единый конфигурационный файл (YAML), логирование через Weights & Biases или MLflow;  
– Код: `rlhf_core.py`, `reward_modeling.py`, `preference_collection.py`.

### **3.4. Обучение и мониторинг с контролем качества**

**Задача:** Обеспечить стабильность и интерпретируемость обучения.  
**Требования:**  
– Для RM: валидация по AUC и ранговой корреляции (Spearman);  
– Для PPO:  
 ✓ Контроль KL ≤ 0.02,  
 ✓ Ограничение количества шагов (≤200),  
 ✓ Мониторинг энтропии и дисперсии advantage;  
– Для DPO: подбор β через валидацию;  
– Сохранение: checkpoint’ов, логов, embedding-проекций (UMAP по скрытым состояниям).

### **3.5. Многоаспектная оценка качества**

**Задача:** Избежать «метрического иллюзионизма».  
**Методы:**  
– **Автоматические**: BERTScore, MAUVE, Self-BLEU (для разнообразия), FactScore (для правдивости);  
– **Экспертные**:  
 ✓ 100 случайных пар оцениваются 3 независимыми экспертами по 5-балльной шкале,  
 ✓ A/B-тест на живой аудитории (≥200 решений);  
– **Поведенческие**:  
 ✓ Частота триггерных фраз («рад помочь», «конечно!»),  
 ✓ Средняя длина, лексическая сложность (MTLD),  
 ✓ Анализ отказов от ответа («я не могу…»).

### **3.6. Анализ смещений и этических рисков**

**Задача:** Выявить и документировать нежелательные эффекты.  
**Требования:**  
– Тест на **культурное смещение**: сравнение ответов на нейтральные запросы по гендеру/национальному признаку;  
– Тест на **избегание**: доля ответов с отказом на сложные темы (политика, религия);  
– Анализ **переоптимизации**: корреляция RM-оценки и экспертной оценки (ожидается нелинейный спад);  
– Формирование **этического отчёта** (Ethics Appendix).

### **3.7. Интерпретация поведенческих сдвигов**

**Задача:** Понять, *как именно* модель меняет своё поведение.  
**Методы:**  
– Сравнение распределений n-грамм до/после;  
– Анализ активаций в ключевых слоях (через `Captum`);  
– Качественные кейсы:  
 ✓ «Модель начала избегать прямых ответов»,  
 ✓ «Увеличилась доля общих фраз при снижении фактологической точности».

### **3.8. Разработка платформы для online RLHF**

**Задача:** Создать инструмент для непрерывного улучшения.  
**Функционал:**  
– Ввод запроса;  
– Слепой A/B-тест между 2–3 моделями;  
– Оценка по 3 критериям + комментарий;  
– Автоматическое сохранение в HF Dataset;  
– Возможность ручного триггера дообучения RM.  
**Технологии:** Gradio + Hugging Face Inference Endpoints + Datasets.

### **3.9. Публикация и стандарты открытой науки**

**Задача:** Обеспечить максимальную воспроизводимость.  
**Требования:**  
– Репозиторий: MIT-лицензия, Dockerfile, `requirements.txt`, скрипты репродюсинга;  
– Hugging Face:  
 ✓ Dataset: `username/rush-rlhf-preferences`,  
 ✓ Models: `username/ru-rm-base`, `username/ru-rlhf-dpo`, `username/tt-rlhf-ppo`,  
 ✓ Spaces: интерактивный A/B-тест;  
– Model Card: описание критериев, смещений, ограничений, лицензий, этических рисков;  
– Данные на татарском — с указанием источников и разрешений.

---

## **4. Дополнительные исследовательские задания (уровень публикации)**

1. **Constitutional AI на русском**: сгенерируйте пары с помощью правила («ответ должен быть правдивым»), обучите RM без человека.  
2. **Перенос RM между языками**: применим ли русский RM к татарским ответам?  
3. **Self-Play RLHF**: модель генерирует пары, оценивает сама, человек корректирует топ-ошибки.  
4. **Анализ голов внимания в RM**: какие слои отвечают за безопасность, а какие — за полезность?  
5. **Экономика RLHF**: оценка CO₂ и стоимости обучения PPO vs DPO (через CodeCarbon, AWS Calculator).  
6. **RLHF + RAG**: обучите модель генерировать ответы, согласованные с извлечённым контекстом.  
7. **Дефузия предпочтений**: можно ли декомпозировать RM на «стиль» и «содержание»?  
8. **Калибровка RM**: используйте Platt scaling для согласования оценок RM с вероятностями экспертного выбора.

---

## **5. Требования к отчёту**

Отчёт оформляется по **ГОСТ 7.32–2017** и должен содержать:  
1. **Введение** — проблема alignment, ограничения SFT, роль RLHF, обзор PPO/DPO/IPO;  
2. **Методология** — описание конвейера, критериев, архитектур, метрик, этического протокола;  
3. **Результаты** — таблицы, графики обучения, MAUVE, UMAP, кейсы до/после, результаты A/B-теста;  
4. **Обсуждение** — анализ компромиссов, смещений, сравнение с SOTA, ограничения;  
5. **Заключение** — рекомендации по применению RLHF в low-resource и multilingual средах;  
6. **Список источников** — по ГОСТ Р 7.0.5–2008, включая ключевые работы (Christiano et al. 2017, Rafailov et al. 2023);  
7. **Приложения**:  
 – Инструкция для аннотаторов,  
 – Примеры предпочтений,  
 – Ethics Appendix,  
 – Скриншоты интерфейса.  

**Обязательные ссылки:**  
– GitHub-репозиторий,  
– Hugging Face Dataset,  
– Hugging Face Models (с Model Card),  
– Hugging Face Space.

---

## **6. Критерии оценивания**

| Оценка | Критерии |
|--------|---------|
| **Отлично** | Реализованы ≥3 метода (PPO, DPO, IPO); собран многокритериальный набор ≥7K пар с верификацией согласия; обучена multi-head RM; проведён многоаспектный анализ (автоматика + эксперты + поведение); реализован online RLHF; опубликованы все компоненты с Ethics Appendix; отчёт содержит обсуждение смещений, переоптимизации и рекомендации для low-resource языков. |
| **Хорошо** | Реализованы PPO и DPO; набор ≥3K; базовая RM; сравнение по автоматическим и экспертным метрикам; отчёт с визуализациями и обсуждением рисков. |
| **Удовлетворительно** | Реализован DPO на публичных или синтетических данных; базовое сравнение; отчёт с описанием метода. |
| **Неудовлетворительно** | Отсутствует сбор предпочтений или обучение RM; нет сравнения с SFT; не учтены этические аспекты. |
